This notebook displays an example of how to run the mini-grid optimization for multiple settlements at once. It does so using an example settlement file from the Global Electrification Platform (electrifynow.energydata.info) for Sierra Leone.

# Import neccessary packages

In [1]:
from src.mgoptimization.hybrids import *
from src.mgoptimization.pso import *
import os
import numpy as np
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

# Import settlements

Data is retrieved from the example_input folder. Next, 100 sample settlements with more than 1000 people are selected from the file.

In [2]:
df = pd.read_csv('input/sl-2-country-inputs.csv')
df = df.loc[df['PopStartYear'] > 1000]
df = df.sample(100)

# Retrieve solar resource data for the area

Select where to save the PV data

In [3]:
pv_data_folder = 'pv'

Enter your renewables.ninja token

In [4]:
token = ''

This below cells retrieves the PV data from renewables.ninja using your token.

PV data is not retrieved for each location. Rather, it is retrieved from a set of evenly spaced points within the study area made up of the settlements to be evaluated. Then, the PV data from the closest point is used when optimizing the mini-grids. This approach is used for two reasons:
* To reduce the number of files saved on the computer and associated storage space
* Because Renewables.ninja has allows maximum 50 downloads/hour from the API.

If the API limit is reached, the download will pause for 60 minutes and then proceed. 

In [5]:
resolution = 0.5  # This is the resolution between points, measured in degrees. A finer resolution will increase detail as well as time to retrieve input files.

In [6]:
def roundPartial (value, resolution):
    return round (value / resolution) * resolution

max_lat = roundPartial(df['Y_deg'].max(), resolution)
min_lat = roundPartial(df['Y_deg'].min(), resolution)

max_long = roundPartial(df['X_deg'].max(), resolution)
min_long = roundPartial(df['X_deg'].min(), resolution)

lats = np.arange(min_lat, max_lat + resolution, resolution) #.tolist()
longs = np.arange(min_long, max_long + resolution, resolution) #.tolist()

print('{} data points will be retrieved'.format(len(lats) * len(longs)))

42 data points will be retrieved


In [7]:
for lat in lats:
    for long in longs:
        output_file = os.path.join(pv_data_folder, 'pv_data_lat_{}_long_{}.csv'.format(lat, long))

        if os.path.exists(output_file) == False:
            get_pv_data(lat, long, token, pv_data_folder)

#  Run the optimization to find the best combination of PV, diesel and batteries, as well as the corresponding LCOE

The LCOE of generation, PV capacity (kW), diesel capacity (kW), battery capacity (kWh) and Net Present Cost - NPC (USD) are printed and saved to the Multiple_settlements_example.csv file in the outputs folder.

In [8]:
def run_pso(annual_demand, tier, lat, long):
    hourly_ghi, hourly_temp = read_environmental_data(os.path.join(pv_data_folder, 'pv_data_lat_{}_long_{}.csv'.format(lat, long)))

    load_curve = calc_load_curve(tier, annual_demand)

    result = optimizer(diesel_price=0.5,
                   hourly_ghi=hourly_ghi,
                   hourly_temp=hourly_temp,
                   load_curve=load_curve,
                   start_year=2024,
                   end_year=2034,
                   discount_rate=0.08,
                   diesel_cost=378,  # diesel generator capital cost, USD/kW rated power
                   battery_cost=589,  # battery capital capital cost, USD/kWh of storage capacity
                   full_life_cycles=2000, #full battery cycles before failure
                   battery_inverter_cost=608,  # USD/kW
                   pv_cost=1147,  # PV panel capital cost, USD/kW peak power
                   pv_inverter=0, # PV inverter cost, USD/kW peak power, set to 0 if already included in pv_cost
                   charge_controller=0, # PV charge controller cost, USD/kW peak power, set to 0 if already included in pv_cost
                   diesel_limit=0.5, # Maximum share of generation over a year coming from diesel generator
                   lpsp_max=0.005,  # maximum loss of load allowed over the year
                   iterations=100,  # PSO parameter. Reducing will increase spee, but reduce chance of finding global optimum
                   n_particles=100, # PSO parameter. Reducing will increase spee, but reduce chance of finding global optimum
                   verbose=False,
                   options={'c1': 0.8, 'c2': 1, 'w': 0.6})

    lcoe = round(result['gen_lcoe'], 3)
    pv_capacity = round(result['pv_capacity'], 1)
    diesel_capacity = round(result['diesel_gen_capacity'], 1)
    battery_capacity = round(result['battery_capacity'], 1)
    npc = int(round(result['npc'], 0))

    return lcoe, pv_capacity, diesel_capacity, battery_capacity, npc

In [9]:
tier = 3

df['MiniGridLCOE'], df['PVCapacity'], df['DieselCapacity'], df['BatteryCapacity'], df['NPC'] = zip(*df.progress_apply(lambda row: run_pso(
    row['ResidentialDemandTier{}'.format(tier)] * row['PopStartYear'], 
    tier, 
    roundPartial(row['Y_deg'], resolution), 
    roundPartial(row['X_deg'], resolution)), axis=1))

df.to_csv(r'output/Multiple_settlements_example.csv')

df[['id', 'X_deg', 'Y_deg', 'GHI', 'PopStartYear', 'MiniGridLCOE', 'PVCapacity', 'DieselCapacity', 'BatteryCapacity', 'NPC']].sample(10)


  0%|          | 0/100 [00:00<?, ?it/s]

,id,X_deg,Y_deg,GHI,PopStartYear,MiniGridLCOE,PVCapacity,DieselCapacity,BatteryCapacity,NPC
19732,19777,-13.00998,8.95820,1878.02478,1398.422138,0.269,107.2,41.4,92.0,483550
9847,9852,-12.66992,8.45657,1831.79059,1643.801789,0.284,145.5,49.0,116.0,604556
18142,18184,-12.74474,9.12750,1889.02622,2198.607255,0.277,168.1,65.2,161.9,783109
19761,19806,-13.16943,8.91400,1898.00000,1198.647554,0.269,93.2,35.4,78.6,415606
14449,14480,-10.90110,7.92308,1855.00000,1374.065000,0.252,98.3,38.9,80.9,445719
18222,18264,-13.07531,9.00328,1866.78556,2511.452014,0.273,211.2,73.9,161.1,884897
16527,16559,-11.61929,7.52380,1785.00000,2731.283543,0.279,183.4,89.2,220.7,976642
19051,19094,-10.79491,8.07920,1868.14044,2053.100082,0.250,141.1,58.3,121.6,660373
15880,15912,-12.27542,7.66752,1779.14709,1021.221739,0.302,99.3,28.5,112.1,419341
20528,20573,-11.54418,7.27910,1759.61302,1272.461350,0.262,105.5,36.0,74.9,431559
